In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate , MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
) 

prompt = ChatPromptTemplate.from_messages([
    ("system" , "You are a helpful AI talking to human"),
    MessagesPlaceholder(variable_name="chat_history"), 
    ("human" , "{question}")
])

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)


In [3]:
chain.predict(question="My name is kajj8808")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to human
Human: My name is kajj8808

> Finished chain.


'Hello kajj8808! How can I assist you today?'

In [4]:
chain.predict(question="I live in Korea")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to human
Human: My name is kajj8808
AI: Hello kajj8808! How can I assist you today?
Human: I live in Korea

> Finished chain.


"That's great to know! How can I assist you with anything related to Korea or any other topic?"

In [5]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to human
Human: My name is kajj8808
AI: Hello kajj8808! How can I assist you today?
Human: I live in Korea
AI: That's great to know! How can I assist you with anything related to Korea or any other topic?
Human: What is my name?

> Finished chain.


'Your name is kajj8808. How can I assist you today, kajj8808?'

In [6]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='My name is kajj8808'),
  AIMessage(content='Hello kajj8808! How can I assist you today?'),
  HumanMessage(content='I live in Korea'),
  AIMessage(content="That's great to know! How can I assist you with anything related to Korea or any other topic?"),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is kajj8808. How can I assist you today, kajj8808?')]}